v3.3 code optimization, output print fixed

In [ ]:
import pandas as pd
import numpy as np
from numpy import array
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Input, Concatenate
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import time
from google.colab import drive


# parameters
scenario_length = 1000
num_attack_types = 3
num_quad_features = 10

num_quad = 4
num_scenarios = 30

# hyper-parameter
input_lstm = 64
shared_lstm = 256
seq_len = 100
seq_overlap = seq_len - 1

epoch_val = 50
batch_size_val = 128

average_approach = 'macro'

# 1 data preprocessing
drive.mount('/content/drive')
dataset_path = '/content/drive/My Drive/code/dataset/network/dataset.csv'
data = pd.read_csv(dataset_path)

# drop time
data = data.drop(columns=['time'])

# combine network labels together
labels = pd.DataFrame()

# Function to create identification vector
def create_identification_vector(row):
    attack_types = [0, 0, 0]  # Initial vector (no attack)
    for col_idx in [51, 54, 57, 60, 62][:num_quad]:
        if row.iloc[col_idx] > 0:
            attack_type_idx = int(row.iloc[col_idx]) - 1
            attack_types[attack_type_idx] = 1  # Mark the correct attack type as 1
    return attack_types

# Dynamically handle labels and targets based on num_quad
cols_to_check = [50, 53, 56, 59, 62][:num_quad]  # Select relevant columns
labels['label'] = data.iloc[:, cols_to_check].apply(lambda row: any(row == 1), axis=1).astype(int)
labels['type'] = data.apply(create_identification_vector, axis=1)
labels['target'] = data.iloc[:, cols_to_check].apply(lambda row: row.astype(int).tolist(), axis=1)

# remove other quads data and old labels
data = data.iloc[:, :num_quad * 10]

# 1.1 normalization
scaler = StandardScaler()
data_normalized = scaler.fit_transform(data)
data_normalized = pd.DataFrame(data_normalized, columns=data.columns)
data_normalized = pd.concat([data_normalized, labels], axis=1)

# 1.2 sequence generation
sequences = []
step_len = seq_len - seq_overlap

for s in range(0, num_scenarios):
    scenario_start = s * scenario_length
    for i in range(scenario_start, scenario_start + scenario_length - seq_len, step_len):
        sequence = data_normalized[i:i + seq_len]
        sequences.append(sequence)
data_sequences = array(sequences)

# 2 data preperation
# 2.1 train-test split
data_reshaped = data_sequences.reshape(data_sequences.shape[0], -1)
X_train, X_test = train_test_split(data_reshaped, test_size=0.25, random_state=42)
X_train = X_train.reshape(X_train.shape[0], data_sequences.shape[1], data_sequences.shape[2])
X_test = X_test.reshape(X_test.shape[0], data_sequences.shape[1], data_sequences.shape[2])


# 2.2 reshape data for LSTM network
y_train_detection = np.array(X_train[:, -1, -3].reshape(-1, 1), dtype=np.float32)
y_train_identification = np.array(X_train[:, -1, -2].tolist(), dtype=np.float32)
y_train_isolation = np.array(X_train[:, -1, -1].tolist(), dtype=np.float32)
X_train = np.array(X_train[:, :, :-3], dtype=np.float32)

y_test_detection = np.array(X_test[:, -1, -3].reshape(-1, 1), dtype=np.float32)
y_test_identification = np.array(X_test[:, -1, -2].tolist(), dtype=np.float32)
y_test_isolation = np.array(X_test[:, -1, -1].tolist(), dtype=np.float32)
X_test = np.array(X_test[:, :, :-3], dtype=np.float32)


# Split X_train into seperate part for different input heads for each quadcopter
X_train_heads = []
X_test_heads = []

for i in range(num_quad):
    X_train_heads.append(X_train[:, :, i*10:(i+1)*10])
    X_test_heads.append(X_test[:, :, i*10:(i+1)*10])

# 3. Model creation
# 3.1 Model architecture
inputs = []
lstm_layers = []

# Create input and LSTM layers based on num_quad
for i in range(num_quad):
    input_layer = Input(shape=(seq_len, 10))
    lstm_layer = LSTM(input_lstm, return_sequences=True)(input_layer)
    inputs.append(input_layer)
    lstm_layers.append(lstm_layer)

# Shared LSTM layer
concat = Concatenate()(lstm_layers)
shared_lstm = LSTM(shared_lstm)(concat)

# Outputs
output_detection = Dense(1, activation='sigmoid', name='detection_output')(shared_lstm)
output_identification = Dense(num_attack_types, activation='sigmoid', name='identification_output')(shared_lstm)
output_isolation = Dense(num_quad, activation='sigmoid', name='isolation_output')(shared_lstm)

# Model
model = Model(inputs=inputs, outputs=[output_detection, output_identification, output_isolation])


# 3.2 model compile
model.compile(
  loss={
      'detection_output': 'binary_crossentropy',
      'identification_output': 'binary_crossentropy',
      'isolation_output': 'binary_crossentropy'
  },
  optimizer='adam',
  metrics={
      'detection_output': ['accuracy'],
      'identification_output': ['accuracy'],
      'isolation_output': ['accuracy']
  }
)

print(model.summary())


# 3.3 model train
start_time = time.time()
model.fit(X_train_heads,
         [y_train_detection, y_train_identification, y_train_isolation],
         epochs=epoch_val, batch_size=batch_size_val)
end_time = time.time()


# 3.4 model test
y_pred_detection, y_pred_identification, y_pred_isolation =\
model.predict(X_test_heads)

# Convert predictions for binary and multi-class
y_pred_detection = (y_pred_detection > 0.5).astype(int).reshape(-1)
y_pred_identification = (y_pred_identification > 0.5).astype(int)
y_pred_isolation = (y_pred_isolation > 0.5).astype(int)


# 3.5 print results for each column
training_time = end_time - start_time
print(f"Training Time: {training_time:.1f}")


# Calculate metrics for detection
accuracy_det = accuracy_score(y_test_detection, y_pred_detection)
precision_det = precision_score(y_test_detection, y_pred_detection)
recall_det = recall_score(y_test_detection, y_pred_detection)
f1_det = f1_score(y_test_detection, y_pred_detection)

# Calculate accuracy for identification
accuracy_iden = accuracy_score(y_test_identification, y_pred_identification)
precision_iden = precision_score(y_test_identification, y_pred_identification, average=average_approach)
recall_iden = recall_score(y_test_identification, y_pred_identification, average=average_approach)
f1_iden = f1_score(y_test_identification, y_pred_identification, average=average_approach)

# Calculate accuracy for isolation
accuracy_iso = accuracy_score(y_test_isolation, y_pred_isolation)
precision_iso = precision_score(y_test_isolation, y_pred_isolation, average=average_approach)
recall_iso = recall_score(y_test_isolation, y_pred_isolation, average=average_approach)
f1_iso = f1_score(y_test_isolation, y_pred_isolation, average=average_approach)


# Print results
print('Detection:')
print(f'    Accuracy: {accuracy_det:.3f}')
print(f'    Precision: {precision_det:.3f}')
print(f'    Recall: {recall_det:.3f}')
print(f'    F1-score: {f1_det:.3f}')

print('Identification:')
print(f'    Accuracy: {accuracy_iden:.3f}')
print(f'    Precision: {precision_iden:.3f}')
print(f'    Recall: {recall_iden:.3f}')
print(f'    F1-score: {f1_iden:.3f}')

print('Isolation:')
print(f'    Accuracy: {accuracy_iso:.3f}')
print(f'    Precision: {precision_iso:.3f}')
print(f'    Recall: {recall_iso:.3f}')
print(f'    F1-score: {f1_iso:.3f}')

# print('Classification Report:')
# print(classification_report(y_test_detection, y_pred_detection))
# print(classification_report(y_test_identification, y_pred_identification))
# print(classification_report(y_test_isolation, y_pred_isolation))

v2 (update for 3-5 quads)

In [ ]:
import pandas as pd
import numpy as np
from numpy import array
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Input, Concatenate
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from google.colab import drive
import time

# parameters
scenario_length = 1000
num_attack_types = 3 + 1
num_attack_targets = 10 + 1
num_quad_features = 10

num_quad = 5
num_scenarios = 10

# hyper-parameter
seq_len = 40
seq_overlap = seq_len - 1
epoch_val = 5
batch_size_val = 128

# 1 data preprocessing
drive.mount('/content/drive')
dataset_path = '/content/drive/My Drive/code/dataset/network/dataset.csv'
data = pd.read_csv(dataset_path)

# drop time
data = data.drop(columns=['time'])

# remove other quads data
if num_quad == 2:
  data = data.iloc[:, list(range(20)) + list(range(50, 56))]
if num_quad == 3:
  data = data.iloc[:, list(range(30)) + list(range(50, 59))]
if num_quad == 4:
  data = data.iloc[:, list(range(40)) + list(range(50, 62))]
if num_quad == 5:
  data = data.iloc[:, list(range(50)) + list(range(50, 65))]

# 1.1 normalization
if num_quad == 2:
  labels = data[data.columns[20:26]]
  data.drop(columns=data.columns[20:26], inplace=True)
if num_quad == 3:
  labels = data[data.columns[30:39]]
  data.drop(columns=data.columns[30:39], inplace=True)
if num_quad == 4:
  labels = data[data.columns[40:52]]
  data.drop(columns=data.columns[40:52], inplace=True)
if num_quad == 5:
  labels = data[data.columns[50:65]]
  data.drop(columns=data.columns[50:65], inplace=True)

scaler = StandardScaler()
data_normalized = scaler.fit_transform(data)
data_normalized = pd.DataFrame(data_normalized, columns=data.columns)
data_normalized = pd.concat([data_normalized, labels], axis=1)

# 1.2 sequence generation
sequences = []
step_len = seq_len - seq_overlap

for s in range(0, num_scenarios):
    scenario_start = s * scenario_length
    for i in range(scenario_start, scenario_start + scenario_length - seq_len, step_len):
        sequence = data_normalized[i:i + seq_len]
        sequences.append(sequence)
data_sequences = array(sequences)

# 2 data preperation
# 2.1 train-test split
data_reshaped = data_sequences.reshape(data_sequences.shape[0], -1)
X_train, X_test = train_test_split(data_reshaped, test_size=0.25, random_state=42)
X_train = X_train.reshape(X_train.shape[0], data_sequences.shape[1], data_sequences.shape[2])
X_test = X_test.reshape(X_test.shape[0], data_sequences.shape[1], data_sequences.shape[2])

# 2.2 reshape data for LSTM network
if num_quad == 2:
  y_train_detection_1 = X_train[:, -1, -6]
  y_train_identification_1 = X_train[:, -1, -5]
  y_train_isolation_1 = X_train[:, -1, -4]
  y_train_detection_2 = X_train[:, -1, -3]
  y_train_identification_2 = X_train[:, -1, -2]
  y_train_isolation_2 = X_train[:, -1, -1]
  X_train = X_train[:, :, :-6]

  y_test_detection_1 = X_test[:, -1, -6]
  y_test_identification_1 = X_test[:, -1, -5]
  y_test_isolation_1 = X_test[:, -1, -4]
  y_test_detection_2 = X_test[:, -1, -3]
  y_test_identification_2 = X_test[:, -1, -2]
  y_test_isolation_2 = X_test[:, -1, -1]
  X_test = X_test[:, :, :-6]

if num_quad == 3:
  y_train_detection_1 = X_train[:, -1, -9]
  y_train_identification_1 = X_train[:, -1, -8]
  y_train_isolation_1 = X_train[:, -1, -7]
  y_train_detection_2 = X_train[:, -1, -6]
  y_train_identification_2 = X_train[:, -1, -5]
  y_train_isolation_2 = X_train[:, -1, -4]
  y_train_detection_3 = X_train[:, -1, -3]
  y_train_identification_3 = X_train[:, -1, -2]
  y_train_isolation_3 = X_train[:, -1, -1]
  X_train = X_train[:, :, :-9]

  y_test_detection_1 = X_test[:, -1, -9]
  y_test_identification_1 = X_test[:, -1, -8]
  y_test_isolation_1 = X_test[:, -1, -7]
  y_test_detection_2 = X_test[:, -1, -6]
  y_test_identification_2 = X_test[:, -1, -5]
  y_test_isolation_2 = X_test[:, -1, -4]
  y_test_detection_3 = X_test[:, -1, -3]
  y_test_identification_3 = X_test[:, -1, -2]
  y_test_isolation_3 = X_test[:, -1, -1]
  X_test = X_test[:, :, :-9]

if num_quad == 4:
  y_train_detection_1 = X_train[:, -1, -12]
  y_train_identification_1 = X_train[:, -1, -11]
  y_train_isolation_1 = X_train[:, -1, -10]
  y_train_detection_2 = X_train[:, -1, -9]
  y_train_identification_2 = X_train[:, -1, -8]
  y_train_isolation_2 = X_train[:, -1, -7]
  y_train_detection_3 = X_train[:, -1, -6]
  y_train_identification_3 = X_train[:, -1, -5]
  y_train_isolation_3 = X_train[:, -1, -4]
  y_train_detection_4 = X_train[:, -1, -3]
  y_train_identification_4 = X_train[:, -1, -2]
  y_train_isolation_4 = X_train[:, -1, -1]
  X_train = X_train[:, :, :-12]

  y_test_detection_1 = X_test[:, -1, -12]
  y_test_identification_1 = X_test[:, -1, -11]
  y_test_isolation_1 = X_test[:, -1, -10]
  y_test_detection_2 = X_test[:, -1, -9]
  y_test_identification_2 = X_test[:, -1, -8]
  y_test_isolation_2 = X_test[:, -1, -7]
  y_test_detection_3 = X_test[:, -1, -6]
  y_test_identification_3 = X_test[:, -1, -5]
  y_test_isolation_3 = X_test[:, -1, -4]
  y_test_detection_4 = X_test[:, -1, -3]
  y_test_identification_4 = X_test[:, -1, -2]
  y_test_isolation_4 = X_test[:, -1, -1]
  X_test = X_test[:, :, :-12]

if num_quad == 5:
  y_train_detection_1 = X_train[:, -1, -15]
  y_train_identification_1 = X_train[:, -1, -14]
  y_train_isolation_1 = X_train[:, -1, -13]
  y_train_detection_2 = X_train[:, -1, -12]
  y_train_identification_2 = X_train[:, -1, -11]
  y_train_isolation_2 = X_train[:, -1, -10]
  y_train_detection_3 = X_train[:, -1, -9]
  y_train_identification_3 = X_train[:, -1, -8]
  y_train_isolation_3 = X_train[:, -1, -7]
  y_train_detection_4 = X_train[:, -1, -6]
  y_train_identification_4 = X_train[:, -1, -5]
  y_train_isolation_4 = X_train[:, -1, -4]
  y_train_detection_5 = X_train[:, -1, -3]
  y_train_identification_5 = X_train[:, -1, -2]
  y_train_isolation_5 = X_train[:, -1, -1]
  X_train = X_train[:, :, :-15]

  y_test_detection_1 = X_test[:, -1, -15]
  y_test_identification_1 = X_test[:, -1, -14]
  y_test_isolation_1 = X_test[:, -1, -13]
  y_test_detection_2 = X_test[:, -1, -12]
  y_test_identification_2 = X_test[:, -1, -11]
  y_test_isolation_2 = X_test[:, -1, -10]
  y_test_detection_3 = X_test[:, -1, -9]
  y_test_identification_3 = X_test[:, -1, -8]
  y_test_isolation_3 = X_test[:, -1, -7]
  y_test_detection_4 = X_test[:, -1, -6]
  y_test_identification_4 = X_test[:, -1, -5]
  y_test_isolation_4 = X_test[:, -1, -4]
  y_test_detection_5 = X_test[:, -1, -3]
  y_test_identification_5 = X_test[:, -1, -2]
  y_test_isolation_5 = X_test[:, -1, -1]
  X_test = X_test[:, :, :-15]


# Split X_train into seperate part for different input heads for each quadcopter
if num_quad == 2:
  X_train_head_1 = X_train[:, :, 0:10]
  X_train_head_2 = X_train[:, :, 10:20]

  X_test_head_1 = X_test[:, :, 0:10]
  X_test_head_2 = X_test[:, :, 10:20]

if num_quad == 3:
  X_train_head_1 = X_train[:, :, 0:10]
  X_train_head_2 = X_train[:, :, 10:20]
  X_train_head_3 = X_train[:, :, 20:30]

  X_test_head_1 = X_test[:, :, 0:10]
  X_test_head_2 = X_test[:, :, 10:20]
  X_test_head_3 = X_test[:, :, 20:30]

if num_quad == 4:
  X_train_head_1 = X_train[:, :, 0:10]
  X_train_head_2 = X_train[:, :, 10:20]
  X_train_head_3 = X_train[:, :, 20:30]
  X_train_head_4 = X_train[:, :, 30:40]

  X_test_head_1 = X_test[:, :, 0:10]
  X_test_head_2 = X_test[:, :, 10:20]
  X_test_head_3 = X_test[:, :, 20:30]
  X_test_head_4 = X_test[:, :, 30:40]

if num_quad == 5:
  X_train_head_1 = X_train[:, :, 0:10]
  X_train_head_2 = X_train[:, :, 10:20]
  X_train_head_3 = X_train[:, :, 20:30]
  X_train_head_4 = X_train[:, :, 30:40]
  X_train_head_5 = X_train[:, :, 40:50]

  X_test_head_1 = X_test[:, :, 0:10]
  X_test_head_2 = X_test[:, :, 10:20]
  X_test_head_3 = X_test[:, :, 20:30]
  X_test_head_4 = X_test[:, :, 30:40]
  X_test_head_5 = X_test[:, :, 40:50]


# 3. model creation
# 3.1 model archiecture
if num_quad == 2:
  # input
  input_1 = Input(shape=(seq_len, 10))
  lstm_1 = LSTM(32, return_sequences=True)(input_1)
  input_2 = Input(shape=(seq_len, 10))
  lstm_2 = LSTM(32, return_sequences=True)(input_2)

  # shared
  concat = Concatenate()([lstm_1, lstm_2])
  shared_lstm = LSTM(64)(concat)

  # output
  output_detection_1 = Dense(1, activation='sigmoid', name='detection_output_1')(shared_lstm)
  output_identification_1 = Dense(num_attack_types, activation='softmax', name='identification_output_1')(shared_lstm)
  output_isolation_1 = Dense(num_attack_targets, activation='softmax', name='isolation_output_1')(shared_lstm)
  output_detection_2 = Dense(1, activation='sigmoid', name='detection_output_2')(shared_lstm)
  output_identification_2 = Dense(num_attack_types, activation='softmax', name='identification_output_2')(shared_lstm)
  output_isolation_2 = Dense(num_attack_targets, activation='softmax', name='isolation_output_2')(shared_lstm)

  model = Model(inputs=[input_1, input_2], outputs=[output_detection_1, output_identification_1, output_isolation_1,
                                                    output_detection_2, output_identification_2, output_isolation_2])

if num_quad == 3:
  # input
  input_1 = Input(shape=(seq_len, 10))
  lstm_1 = LSTM(32, return_sequences=True)(input_1)
  input_2 = Input(shape=(seq_len, 10))
  lstm_2 = LSTM(32, return_sequences=True)(input_2)
  input_3 = Input(shape=(seq_len, 10))
  lstm_3 = LSTM(32, return_sequences=True)(input_3)

  # shared
  concat = Concatenate()([lstm_1, lstm_2, lstm_3])
  shared_lstm = LSTM(64)(concat)

  # output
  output_detection_1 = Dense(1, activation='sigmoid', name='detection_output_1')(shared_lstm)
  output_identification_1 = Dense(num_attack_types, activation='softmax', name='identification_output_1')(shared_lstm)
  output_isolation_1 = Dense(num_attack_targets, activation='softmax', name='isolation_output_1')(shared_lstm)
  output_detection_2 = Dense(1, activation='sigmoid', name='detection_output_2')(shared_lstm)
  output_identification_2 = Dense(num_attack_types, activation='softmax', name='identification_output_2')(shared_lstm)
  output_isolation_2 = Dense(num_attack_targets, activation='softmax', name='isolation_output_2')(shared_lstm)
  output_detection_3 = Dense(1, activation='sigmoid', name='detection_output_3')(shared_lstm)
  output_identification_3 = Dense(num_attack_types, activation='softmax', name='identification_output_3')(shared_lstm)
  output_isolation_3 = Dense(num_attack_targets, activation='softmax', name='isolation_output_3')(shared_lstm)

  model = Model(inputs=[input_1, input_2, input_3], outputs=[output_detection_1, output_identification_1, output_isolation_1,
                                                             output_detection_2, output_identification_2, output_isolation_2,
                                                             output_detection_3, output_identification_3, output_isolation_3])

if num_quad == 4:
    # input
    input_1 = Input(shape=(seq_len, 10))
    lstm_1 = LSTM(32, return_sequences=True)(input_1)
    input_2 = Input(shape=(seq_len, 10))
    lstm_2 = LSTM(32, return_sequences=True)(input_2)
    input_3 = Input(shape=(seq_len, 10))
    lstm_3 = LSTM(32, return_sequences=True)(input_3)
    input_4 = Input(shape=(seq_len, 10))
    lstm_4 = LSTM(32, return_sequences=True)(input_4)

    # shared
    concat = Concatenate()([lstm_1, lstm_2, lstm_3, lstm_4])
    shared_lstm = LSTM(64)(concat)

    # output
    output_detection_1 = Dense(1, activation='sigmoid', name='detection_output_1')(shared_lstm)
    output_identification_1 = Dense(num_attack_types, activation='softmax', name='identification_output_1')(shared_lstm)
    output_isolation_1 = Dense(num_attack_targets, activation='softmax', name='isolation_output_1')(shared_lstm)
    output_detection_2 = Dense(1, activation='sigmoid', name='detection_output_2')(shared_lstm)
    output_identification_2 = Dense(num_attack_types, activation='softmax', name='identification_output_2')(shared_lstm)
    output_isolation_2 = Dense(num_attack_targets, activation='softmax', name='isolation_output_2')(shared_lstm)
    output_detection_3 = Dense(1, activation='sigmoid', name='detection_output_3')(shared_lstm)
    output_identification_3 = Dense(num_attack_types, activation='softmax', name='identification_output_3')(shared_lstm)
    output_isolation_3 = Dense(num_attack_targets, activation='softmax', name='isolation_output_3')(shared_lstm)
    output_detection_4 = Dense(1, activation='sigmoid', name='detection_output_4')(shared_lstm)
    output_identification_4 = Dense(num_attack_types, activation='softmax', name='identification_output_4')(shared_lstm)
    output_isolation_4 = Dense(num_attack_targets, activation='softmax', name='isolation_output_4')(shared_lstm)

    model = Model(inputs=[input_1, input_2, input_3, input_4], outputs=[output_detection_1, output_identification_1, output_isolation_1,
                                                                      output_detection_2, output_identification_2, output_isolation_2,
                                                                      output_detection_3, output_identification_3, output_isolation_3,
                                                                      output_detection_4, output_identification_4, output_isolation_4])

if num_quad == 5:
    # input
    input_1 = Input(shape=(seq_len, 10))
    lstm_1 = LSTM(32, return_sequences=True)(input_1)
    input_2 = Input(shape=(seq_len, 10))
    lstm_2 = LSTM(32, return_sequences=True)(input_2)
    input_3 = Input(shape=(seq_len, 10))
    lstm_3 = LSTM(32, return_sequences=True)(input_3)
    input_4 = Input(shape=(seq_len, 10))
    lstm_4 = LSTM(32, return_sequences=True)(input_4)
    input_5 = Input(shape=(seq_len, 10))
    lstm_5 = LSTM(32, return_sequences=True)(input_5)

    # shared
    concat = Concatenate()([lstm_1, lstm_2, lstm_3, lstm_4, lstm_5])
    shared_lstm = LSTM(64)(concat)

    # output
    output_detection_1 = Dense(1, activation='sigmoid', name='detection_output_1')(shared_lstm)
    output_identification_1 = Dense(num_attack_types, activation='softmax', name='identification_output_1')(shared_lstm)
    output_isolation_1 = Dense(num_attack_targets, activation='softmax', name='isolation_output_1')(shared_lstm)
    output_detection_2 = Dense(1, activation='sigmoid', name='detection_output_2')(shared_lstm)
    output_identification_2 = Dense(num_attack_types, activation='softmax', name='identification_output_2')(shared_lstm)
    output_isolation_2 = Dense(num_attack_targets, activation='softmax', name='isolation_output_2')(shared_lstm)
    output_detection_3 = Dense(1, activation='sigmoid', name='detection_output_3')(shared_lstm)
    output_identification_3 = Dense(num_attack_types, activation='softmax', name='identification_output_3')(shared_lstm)
    output_isolation_3 = Dense(num_attack_targets, activation='softmax', name='isolation_output_3')(shared_lstm)
    output_detection_4 = Dense(1, activation='sigmoid', name='detection_output_4')(shared_lstm)
    output_identification_4 = Dense(num_attack_types, activation='softmax', name='identification_output_4')(shared_lstm)
    output_isolation_4 = Dense(num_attack_targets, activation='softmax', name='isolation_output_4')(shared_lstm)
    output_detection_5 = Dense(1, activation='sigmoid', name='detection_output_5')(shared_lstm)
    output_identification_5 = Dense(num_attack_types, activation='softmax', name='identification_output_5')(shared_lstm)
    output_isolation_5 = Dense(num_attack_targets, activation='softmax', name='isolation_output_5')(shared_lstm)

    model = Model(inputs=[input_1, input_2, input_3, input_4, input_5], outputs=[output_detection_1, output_identification_1, output_isolation_1,
                                                                      output_detection_2, output_identification_2, output_isolation_2,
                                                                      output_detection_3, output_identification_3, output_isolation_3,
                                                                      output_detection_4, output_identification_4, output_isolation_4,
                                                                      output_detection_5, output_identification_5, output_isolation_5])


# 3.2 model compile
if num_quad == 2:
  model.compile(
      loss={
          'detection_output_1': 'binary_crossentropy',
          'identification_output_1': 'sparse_categorical_crossentropy',
          'isolation_output_1': 'sparse_categorical_crossentropy',
          'detection_output_2': 'binary_crossentropy',
          'identification_output_2': 'sparse_categorical_crossentropy',
          'isolation_output_2': 'sparse_categorical_crossentropy'
      },
      optimizer='adam',
      metrics={
          'detection_output_1': ['accuracy'],
          'identification_output_1': ['accuracy'],
          'isolation_output_1': ['accuracy'],
          'detection_output_2': ['accuracy'],
          'identification_output_2': ['accuracy'],
          'isolation_output_2': ['accuracy']
      }
  )

if num_quad == 3:
  model.compile(
      loss={
          'detection_output_1': 'binary_crossentropy',
          'identification_output_1': 'sparse_categorical_crossentropy',
          'isolation_output_1': 'sparse_categorical_crossentropy',
          'detection_output_2': 'binary_crossentropy',
          'identification_output_2': 'sparse_categorical_crossentropy',
          'isolation_output_2': 'sparse_categorical_crossentropy',
          'detection_output_3': 'binary_crossentropy',
          'identification_output_3': 'sparse_categorical_crossentropy',
          'isolation_output_3': 'sparse_categorical_crossentropy'
      },
      optimizer='adam',
      metrics={
          'detection_output_1': ['accuracy'],
          'identification_output_1': ['accuracy'],
          'isolation_output_1': ['accuracy'],
          'detection_output_2': ['accuracy'],
          'identification_output_2': ['accuracy'],
          'isolation_output_2': ['accuracy'],
          'detection_output_3': ['accuracy'],
          'identification_output_3': ['accuracy'],
          'isolation_output_3': ['accuracy']
      }
  )

if num_quad == 4:
  model.compile(
      loss={
          'detection_output_1': 'binary_crossentropy',
          'identification_output_1': 'sparse_categorical_crossentropy',
          'isolation_output_1': 'sparse_categorical_crossentropy',
          'detection_output_2': 'binary_crossentropy',
          'identification_output_2': 'sparse_categorical_crossentropy',
          'isolation_output_2': 'sparse_categorical_crossentropy',
          'detection_output_3': 'binary_crossentropy',
          'identification_output_3': 'sparse_categorical_crossentropy',
          'isolation_output_3': 'sparse_categorical_crossentropy',
          'detection_output_4': 'binary_crossentropy',
          'identification_output_4': 'sparse_categorical_crossentropy',
          'isolation_output_4': 'sparse_categorical_crossentropy'
      },
      optimizer='adam',
      metrics={
          'detection_output_1': ['accuracy'],
          'identification_output_1': ['accuracy'],
          'isolation_output_1': ['accuracy'],
          'detection_output_2': ['accuracy'],
          'identification_output_2': ['accuracy'],
          'isolation_output_2': ['accuracy'],
          'detection_output_3': ['accuracy'],
          'identification_output_3': ['accuracy'],
          'isolation_output_3': ['accuracy'],
          'detection_output_4': ['accuracy'],
          'identification_output_4': ['accuracy'],
          'isolation_output_4': ['accuracy']
      }
  )

if num_quad == 5:
  model.compile(
      loss={
          'detection_output_1': 'binary_crossentropy',
          'identification_output_1': 'sparse_categorical_crossentropy',
          'isolation_output_1': 'sparse_categorical_crossentropy',
          'detection_output_2': 'binary_crossentropy',
          'identification_output_2': 'sparse_categorical_crossentropy',
          'isolation_output_2': 'sparse_categorical_crossentropy',
          'detection_output_3': 'binary_crossentropy',
          'identification_output_3': 'sparse_categorical_crossentropy',
          'isolation_output_3': 'sparse_categorical_crossentropy',
          'detection_output_4': 'binary_crossentropy',
          'identification_output_4': 'sparse_categorical_crossentropy',
          'isolation_output_4': 'sparse_categorical_crossentropy',
          'detection_output_5': 'binary_crossentropy',
          'identification_output_5': 'sparse_categorical_crossentropy',
          'isolation_output_5': 'sparse_categorical_crossentropy'
      },
      optimizer='adam',
      metrics={
          'detection_output_1': ['accuracy'],
          'identification_output_1': ['accuracy'],
          'isolation_output_1': ['accuracy'],
          'detection_output_2': ['accuracy'],
          'identification_output_2': ['accuracy'],
          'isolation_output_2': ['accuracy'],
          'detection_output_3': ['accuracy'],
          'identification_output_3': ['accuracy'],
          'isolation_output_3': ['accuracy'],
          'detection_output_4': ['accuracy'],
          'identification_output_4': ['accuracy'],
          'isolation_output_4': ['accuracy'],
          'detection_output_5': ['accuracy'],
          'identification_output_5': ['accuracy'],
          'isolation_output_5': ['accuracy']
      }
  )


print(model.summary())


# 3.3 model train
start_time = time.time()

if num_quad == 2:
  model.fit([X_train_head_1, X_train_head_2],
            [y_train_detection_1, y_train_identification_1, y_train_isolation_1,
             y_train_detection_2, y_train_identification_2, y_train_isolation_2],
            epochs=epoch_val, batch_size=batch_size_val)

if num_quad == 3:
  model.fit([X_train_head_1, X_train_head_2, X_train_head_3],
            [y_train_detection_1, y_train_identification_1, y_train_isolation_1,
             y_train_detection_2, y_train_identification_2, y_train_isolation_2,
             y_train_detection_3, y_train_identification_3, y_train_isolation_3],
            epochs=epoch_val, batch_size=batch_size_val)

if num_quad == 4:
  model.fit([X_train_head_1, X_train_head_2, X_train_head_3, X_train_head_4],
            [y_train_detection_1, y_train_identification_1, y_train_isolation_1,
              y_train_detection_2, y_train_identification_2, y_train_isolation_2,
              y_train_detection_3, y_train_identification_3, y_train_isolation_3,
              y_train_detection_4, y_train_identification_4, y_train_isolation_4],
            epochs=epoch_val, batch_size=batch_size_val)

if num_quad == 5:
  model.fit([X_train_head_1, X_train_head_2, X_train_head_3, X_train_head_4, X_train_head_5],
            [y_train_detection_1, y_train_identification_1, y_train_isolation_1,
              y_train_detection_2, y_train_identification_2, y_train_isolation_2,
              y_train_detection_3, y_train_identification_3, y_train_isolation_3,
              y_train_detection_4, y_train_identification_4, y_train_isolation_4,
              y_train_detection_5, y_train_identification_5, y_train_isolation_5],
            epochs=epoch_val, batch_size=batch_size_val)


end_time = time.time()


# 3.4 model test
if num_quad == 2:
  y_pred_detection_1, y_pred_identification_1, y_pred_isolation_1, \
  y_pred_detection_2, y_pred_identification_2, y_pred_isolation_2 =\
  model.predict([X_test_head_1, X_test_head_2])

if num_quad == 3:
  y_pred_detection_1, y_pred_identification_1, y_pred_isolation_1, \
  y_pred_detection_2, y_pred_identification_2, y_pred_isolation_2, \
  y_pred_detection_3, y_pred_identification_3, y_pred_isolation_3 =\
  model.predict([X_test_head_1, X_test_head_2, X_test_head_3])

if num_quad == 4:
    y_pred_detection_1, y_pred_identification_1, y_pred_isolation_1, \
    y_pred_detection_2, y_pred_identification_2, y_pred_isolation_2, \
    y_pred_detection_3, y_pred_identification_3, y_pred_isolation_3, \
    y_pred_detection_4, y_pred_identification_4, y_pred_isolation_4 = \
    model.predict([X_test_head_1, X_test_head_2, X_test_head_3, X_test_head_4])

if num_quad == 5:
    y_pred_detection_1, y_pred_identification_1, y_pred_isolation_1, \
    y_pred_detection_2, y_pred_identification_2, y_pred_isolation_2, \
    y_pred_detection_3, y_pred_identification_3, y_pred_isolation_3, \
    y_pred_detection_4, y_pred_identification_4, y_pred_isolation_4, \
    y_pred_detection_5, y_pred_identification_5, y_pred_isolation_5 = \
    model.predict([X_test_head_1, X_test_head_2, X_test_head_3, X_test_head_4, X_test_head_5])


# Convert predictions for binary and multi-class
if num_quad == 2:
  y_pred_detection_1 = (y_pred_detection_1 > 0.5).astype(int).reshape(-1)
  y_pred_identification_1 = np.argmax(y_pred_identification_1, axis=1)
  y_pred_isolation_1 = np.argmax(y_pred_isolation_1, axis=1)
  y_pred_detection_2 = (y_pred_detection_2 > 0.5).astype(int).reshape(-1)
  y_pred_identification_2 = np.argmax(y_pred_identification_2, axis=1)
  y_pred_isolation_2 = np.argmax(y_pred_isolation_2, axis=1)

if num_quad == 3:
  y_pred_detection_1 = (y_pred_detection_1 > 0.5).astype(int).reshape(-1)
  y_pred_identification_1 = np.argmax(y_pred_identification_1, axis=1)
  y_pred_isolation_1 = np.argmax(y_pred_isolation_1, axis=1)
  y_pred_detection_2 = (y_pred_detection_2 > 0.5).astype(int).reshape(-1)
  y_pred_identification_2 = np.argmax(y_pred_identification_2, axis=1)
  y_pred_isolation_2 = np.argmax(y_pred_isolation_2, axis=1)
  y_pred_detection_3 = (y_pred_detection_3 > 0.5).astype(int).reshape(-1)
  y_pred_identification_3 = np.argmax(y_pred_identification_3, axis=1)
  y_pred_isolation_3 = np.argmax(y_pred_isolation_3, axis=1)

if num_quad == 4:
    y_pred_detection_1 = (y_pred_detection_1 > 0.5).astype(int).reshape(-1)
    y_pred_identification_1 = np.argmax(y_pred_identification_1, axis=1)
    y_pred_isolation_1 = np.argmax(y_pred_isolation_1, axis=1)
    y_pred_detection_2 = (y_pred_detection_2 > 0.5).astype(int).reshape(-1)
    y_pred_identification_2 = np.argmax(y_pred_identification_2, axis=1)
    y_pred_isolation_2 = np.argmax(y_pred_isolation_2, axis=1)
    y_pred_detection_3 = (y_pred_detection_3 > 0.5).astype(int).reshape(-1)
    y_pred_identification_3 = np.argmax(y_pred_identification_3, axis=1)
    y_pred_isolation_3 = np.argmax(y_pred_isolation_3, axis=1)
    y_pred_detection_4 = (y_pred_detection_4 > 0.5).astype(int).reshape(-1)
    y_pred_identification_4 = np.argmax(y_pred_identification_4, axis=1)
    y_pred_isolation_4 = np.argmax(y_pred_isolation_4, axis=1)

if num_quad == 5:
    y_pred_detection_1 = (y_pred_detection_1 > 0.5).astype(int).reshape(-1)
    y_pred_identification_1 = np.argmax(y_pred_identification_1, axis=1)
    y_pred_isolation_1 = np.argmax(y_pred_isolation_1, axis=1)
    y_pred_detection_2 = (y_pred_detection_2 > 0.5).astype(int).reshape(-1)
    y_pred_identification_2 = np.argmax(y_pred_identification_2, axis=1)
    y_pred_isolation_2 = np.argmax(y_pred_isolation_2, axis=1)
    y_pred_detection_3 = (y_pred_detection_3 > 0.5).astype(int).reshape(-1)
    y_pred_identification_3 = np.argmax(y_pred_identification_3, axis=1)
    y_pred_isolation_3 = np.argmax(y_pred_isolation_3, axis=1)
    y_pred_detection_4 = (y_pred_detection_4 > 0.5).astype(int).reshape(-1)
    y_pred_identification_4 = np.argmax(y_pred_identification_4, axis=1)
    y_pred_isolation_4 = np.argmax(y_pred_isolation_4, axis=1)
    y_pred_detection_5 = (y_pred_detection_5 > 0.5).astype(int).reshape(-1)
    y_pred_identification_5 = np.argmax(y_pred_identification_5, axis=1)
    y_pred_isolation_5 = np.argmax(y_pred_isolation_5, axis=1)

# 3.5 print results for each column

training_time = end_time - start_time
print(f"Training Time: {training_time:.1f}")


for quad in range(1, num_quad+1):
    # Retrieve the y_test and y_pred variables dynamically based on the quad number
    y_test_detection = eval(f'y_test_detection_{quad}')
    y_pred_detection = eval(f'y_pred_detection_{quad}')
    y_test_identification = eval(f'y_test_identification_{quad}')
    y_pred_identification = eval(f'y_pred_identification_{quad}')
    y_test_isolation = eval(f'y_test_isolation_{quad}')
    y_pred_isolation = eval(f'y_pred_isolation_{quad}')

    # Calculate metrics for detection
    accuracy = accuracy_score(y_test_detection, y_pred_detection)
    precision = precision_score(y_test_detection, y_pred_detection)
    recall = recall_score(y_test_detection, y_pred_detection)
    f1 = f1_score(y_test_detection, y_pred_detection)

    # Calculate accuracy for identification and isolation
    accuracy_iden = accuracy_score(y_test_identification, y_pred_identification)
    accuracy_isol = accuracy_score(y_test_isolation, y_pred_isolation)

    # Print results
    print(f'Quad {quad}:')
    print(' Detection:')
    print(f'    Accuracy: {accuracy:.3f}')
    print(f'    Precision: {precision:.3f}')
    print(f'    Recall: {recall:.3f}')
    print(f'    F1-score: {f1:.3f}')

    print(' Identification:')
    print(f'    Accuracy: {accuracy_iden:.3f}')

    print(' Localization:')
    print(f'    Accuracy: {accuracy_isol:.3f}')

    print(' Classification Report:')
    print(classification_report(y_test_detection, y_pred_detection))
    print(classification_report(y_test_identification, y_pred_identification))
    print(classification_report(y_test_isolation, y_pred_isolation))


v2 (get DS from drive, update to final version, 2 quads, ability to select between all data or only sensors)

In [ ]:
import pandas as pd
import numpy as np
from numpy import array
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Input, Concatenate
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from google.colab import drive
import time


# parameters
num_features = 20
quad_num_features = 10
scenario_length = 1000
num_attack_types = 3 + 1
num_attack_targets = 10 + 1
num_quad = 2

num_scenarios = 20

# hyper-parameter
seq_len = 40
seq_overlap = seq_len - 1
lstm_blocks = 128
epoch_val = 50
batch_size_val = 128

# 1 data preprocessing
drive.mount('/content/drive')
dataset_path = '/content/drive/My Drive/code/dataset/network/dataset.csv'
data = pd.read_csv(dataset_path)

# remove other quads data
data = data.loc[:, data.columns[:21].tolist() + data.columns[51:57].tolist()]


# 1.1 normalization
labels = data[['label_1', 'type_1', 'target_1', 'label_2', 'type_2', 'target_2']]
data = data.drop(columns=['time', 'label_1', 'type_1', 'target_1', 'label_2', 'type_2', 'target_2'])
scaler = StandardScaler()
data_normalized = scaler.fit_transform(data)
data_normalized = pd.DataFrame(data_normalized, columns=data.columns)
data_normalized = pd.concat([data_normalized, labels], axis=1)

# 1.2 sequence generation
sequences = []
step_len = seq_len - seq_overlap

for s in range(0, num_scenarios):
    scenario_start = s * scenario_length
    for i in range(scenario_start, scenario_start + scenario_length - seq_len, step_len):
        sequence = data_normalized[i:i + seq_len]
        sequences.append(sequence)
data_sequences = array(sequences)

# 2 data preperation
# 2.1 train-test split
data_reshaped = data_sequences.reshape(data_sequences.shape[0], -1)
X_train, X_test = train_test_split(data_reshaped, test_size=0.25, random_state=42)
X_train = X_train.reshape(X_train.shape[0], data_sequences.shape[1], data_sequences.shape[2])
X_test = X_test.reshape(X_test.shape[0], data_sequences.shape[1], data_sequences.shape[2])

# 2.2 reshape data for LSTM network
y_train_detection_1 = X_train[:, -1, -6]
y_train_identification_1 = X_train[:, -1, -5]
y_train_isolation_1 = X_train[:, -1, -4]
y_train_detection_2 = X_train[:, -1, -3]
y_train_identification_2 = X_train[:, -1, -2]
y_train_isolation_2 = X_train[:, -1, -1]
X_train = X_train[:, :, :-6]

y_test_detection_1 = X_test[:, -1, -6]
y_test_identification_1 = X_test[:, -1, -5]
y_test_isolation_1 = X_test[:, -1, -4]
y_test_detection_2 = X_test[:, -1, -3]
y_test_identification_2 = X_test[:, -1, -2]
y_test_isolation_2 = X_test[:, -1, -1]
X_test = X_test[:, :, :-6]

# Split X_train into two parts: first half and second half
# set if both actuator and sensor data will be used or only sensor
X_train_head_1 = X_train[:, :, :10]
X_train_head_2 = X_train[:, :, 10:16]

X_test_head_1 = X_test[:, :, :10]
X_test_head_2 = X_test[:, :, 10:16]


# 3. model creation
# 3.1 model archiecture
# input
input_1 = Input(shape=(seq_len, quad_num_features))
lstm_1 = LSTM(32, return_sequences=True)(input_1)
input_2 = Input(shape=(seq_len, 6))
lstm_2 = LSTM(32, return_sequences=True)(input_2)

# shared
concat = Concatenate()([lstm_1, lstm_2])
shared_lstm = LSTM(64)(concat)

# output
output_detection_1 = Dense(1, activation='sigmoid', name='detection_output_1')(shared_lstm)
output_identification_1 = Dense(num_attack_types, activation='softmax', name='identification_output_1')(shared_lstm)
output_isolation_1 = Dense(num_attack_targets, activation='softmax', name='isolation_output_1')(shared_lstm)
output_detection_2 = Dense(1, activation='sigmoid', name='detection_output_2')(shared_lstm)
output_identification_2 = Dense(num_attack_types, activation='softmax', name='identification_output_2')(shared_lstm)
output_isolation_2 = Dense(num_attack_targets, activation='softmax', name='isolation_output_2')(shared_lstm)

model = Model(inputs=[input_1, input_2], outputs=[output_detection_1, output_identification_1, output_isolation_1,
                                           output_detection_2, output_identification_2, output_isolation_2])

# 3.2 model compile
model.compile(
    loss={
        'detection_output_1': 'binary_crossentropy',
        'identification_output_1': 'sparse_categorical_crossentropy',
        'isolation_output_1': 'sparse_categorical_crossentropy',
        'detection_output_2': 'binary_crossentropy',
        'identification_output_2': 'sparse_categorical_crossentropy',
        'isolation_output_2': 'sparse_categorical_crossentropy'
    },
    optimizer='adam',
    metrics={
        'detection_output_1': ['accuracy'],
        'identification_output_1': ['accuracy'],
        'isolation_output_1': ['accuracy'],
        'detection_output_2': ['accuracy'],
        'identification_output_2': ['accuracy'],
        'isolation_output_2': ['accuracy']
    }
)
print(model.summary())

# 3.3 model train
start_time = time.time()
model.fit([X_train_head_1, X_train_head_2], [y_train_detection_1, y_train_identification_1, y_train_isolation_1,
                    y_train_detection_2, y_train_identification_2, y_train_isolation_2],
                    epochs=epoch_val, batch_size=batch_size_val)
end_time = time.time()


# 3.4 model test
y_pred_detection_1, y_pred_identification_1, y_pred_isolation_1, \
y_pred_detection_2, y_pred_identification_2, y_pred_isolation_2 = model.predict([X_test_head_1, X_test_head_2])

# Convert predictions for binary and multi-class
y_pred_detection_1 = (y_pred_detection_1 > 0.5).astype(int).reshape(-1)
y_pred_identification_1 = np.argmax(y_pred_identification_1, axis=1)
y_pred_isolation_1 = np.argmax(y_pred_isolation_1, axis=1)
y_pred_detection_2 = (y_pred_detection_2 > 0.5).astype(int).reshape(-1)
y_pred_identification_2 = np.argmax(y_pred_identification_2, axis=1)
y_pred_isolation_2 = np.argmax(y_pred_isolation_2, axis=1)

# 3.5 print results for each column

training_time = end_time - start_time

# quad 1
accuracy = accuracy_score(y_test_detection_1, y_pred_detection_1)
precision = precision_score(y_test_detection_1, y_pred_detection_1)
recall = recall_score(y_test_detection_1, y_pred_detection_1)
f1 = f1_score(y_test_detection_1, y_pred_detection_1)
accuracy_iden = accuracy_score(y_test_identification_1, y_pred_identification_1)
accuracy_isol = accuracy_score(y_test_isolation_1, y_pred_isolation_1)

print(f"Training Time: {training_time:.1f}")

print('Quad 1:')
print(' Detection:')
print(f'    Accuracy: {accuracy:.3f}')
print(f'    Precision: {precision:.3f}')
print(f'    Recall: {recall:.3f}')
print(f'    F1-score: {f1:.3f}')

print(' Identification:')
print(f'    Accuracy: {accuracy_iden:.3f}')

print(' Localization:')
print(f'    Accuracy: {accuracy_isol:.3f}')

print(' Classification Report: ')
print(classification_report(y_test_detection_1, y_pred_detection_1))
print(classification_report(y_test_identification_1, y_pred_identification_1))
print(classification_report(y_test_isolation_1, y_pred_isolation_1))

# quad 2
accuracy = accuracy_score(y_test_detection_2, y_pred_detection_2)
precision = precision_score(y_test_detection_2, y_pred_detection_2)
recall = recall_score(y_test_detection_2, y_pred_detection_2)
f1 = f1_score(y_test_detection_2, y_pred_detection_2)
accuracy_iden = accuracy_score(y_test_identification_2, y_pred_identification_2)
accuracy_isol = accuracy_score(y_test_isolation_2, y_pred_isolation_2)

print('Quad 2:')
print(' Detection:')
print(f'    Accuracy: {accuracy:.3f}')
print(f'    Precision: {precision:.3f}')
print(f'    Recall: {recall:.3f}')
print(f'    F1-score: {f1:.3f}')

# 3.6 print results for identification
print(' Identification:')
print(f'    Accuracy: {accuracy_iden:.3f}')

# 3.7 print results for localization
print(' Localization:')
print(f'    Accuracy: {accuracy_isol:.3f}')

print(' Classification Report: ')
print(classification_report(y_test_detection_2, y_pred_detection_2))
print(classification_report(y_test_identification_2, y_pred_identification_2))
print(classification_report(y_test_isolation_2, y_pred_isolation_2))

multi input - multi output LSTM

In [ ]:
import pandas as pd
import numpy as np
from numpy import array
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Input, Concatenate
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from google.colab import drive


# parameters
num_features = 20
quad_num_features = 10
num_scenarios = 6
scenario_length = 1000
attack_type = 0 		# 0: all, 1: DoS, 2: FDI, 3: Replay
num_attack_types = 3 + 1
num_attack_targets = 6 + 1
num_quad = 2

# hyper-parameter
seq_len = 20
seq_overlap = seq_len - 1
lstm_blocks = 64
epoch_val = 20
batch_size_val = 4

# 1 data preprocessing
drive.mount('/content/drive')
dataset_path = '/content/drive/My Drive/code/dataset/network/dataset.csv'
data = pd.read_csv(dataset_path)

# 1.1 normalization
labels = data[['label_1', 'type_1', 'target_1', 'label_2', 'type_2', 'target_2']]
data = data.drop(columns=['time', 'label_1', 'type_1', 'target_1', 'label_2', 'type_2', 'target_2'])
scaler = StandardScaler()
data_normalized = scaler.fit_transform(data)
data_normalized = pd.DataFrame(data_normalized, columns=data.columns)
data_normalized = pd.concat([data_normalized, labels], axis=1)

# 1.2 sequence generation
sequences = []
step_len = seq_len - seq_overlap

if attack_type == 0:
    scenario_range = range(0, 6)
elif attack_type == 1:
    scenario_range = range(0, 2)
elif attack_type == 2:
    scenario_range = range(2, 4)
elif attack_type == 3:
    scenario_range = range(4, 6)

for s in scenario_range:
    scenario_start = s * scenario_length
    for i in range(scenario_start, scenario_start + scenario_length - seq_len, step_len):
        sequence = data_normalized[i:i + seq_len]
        sequences.append(sequence)
data_sequences = array(sequences)

# 2 data preperation
# 2.1 train-test split
data_reshaped = data_sequences.reshape(data_sequences.shape[0], -1)
X_train, X_test = train_test_split(data_reshaped, test_size=0.25, random_state=42)
X_train = X_train.reshape(X_train.shape[0], data_sequences.shape[1], data_sequences.shape[2])
X_test = X_test.reshape(X_test.shape[0], data_sequences.shape[1], data_sequences.shape[2])

# 2.2 reshape data for LSTM network
y_train_detection_1 = X_train[:, -1, -6]
y_train_identification_1 = X_train[:, -1, -5]
y_train_isolation_1 = X_train[:, -1, -4]
y_train_detection_2 = X_train[:, -1, -3]
y_train_identification_2 = X_train[:, -1, -2]
y_train_isolation_2 = X_train[:, -1, -1]
X_train = X_train[:, :, :-6]

y_test_detection_1 = X_test[:, -1, -6]
y_test_identification_1 = X_test[:, -1, -5]
y_test_isolation_1 = X_test[:, -1, -4]
y_test_detection_2 = X_test[:, -1, -3]
y_test_identification_2 = X_test[:, -1, -2]
y_test_isolation_2 = X_test[:, -1, -1]
X_test = X_test[:, :, :-6]

# Split X_train into two parts: first half and second half
X_train_head_1 = X_train[:, :, :10]
X_train_head_2 = X_train[:, :, 10:]
X_test_head_1 = X_test[:, :, :10]
X_test_head_2 = X_test[:, :, 10:]

# 3. model creation
# 3.1 model archiecture
input_own = Input(shape=(seq_len, quad_num_features))
lstm_own = LSTM(16, return_sequences=True)(input_own)
input_neighbor = Input(shape=(seq_len, quad_num_features))
lstm_neighbor = LSTM(16, return_sequences=True)(input_neighbor)

concat = Concatenate()([lstm_own, lstm_neighbor])
shared_lstm = LSTM(16)(concat)

output_detection_1 = Dense(1, activation='sigmoid', name='detection_output_1')(shared_lstm)
output_identification_1 = Dense(num_attack_types, activation='softmax', name='identification_output_1')(shared_lstm)
output_isolation_1 = Dense(num_attack_targets, activation='softmax', name='isolation_output_1')(shared_lstm)
output_detection_2 = Dense(1, activation='sigmoid', name='detection_output_2')(shared_lstm)
output_identification_2 = Dense(num_attack_types, activation='softmax', name='identification_output_2')(shared_lstm)
output_isolation_2 = Dense(num_attack_targets, activation='softmax', name='isolation_output_2')(shared_lstm)

model = Model(inputs=[input_own, input_neighbor], outputs=[output_detection_1, output_identification_1, output_isolation_1,
                                           output_detection_2, output_identification_2, output_isolation_2])

# 3.2 model compile
model.compile(
    loss={
        'detection_output_1': 'binary_crossentropy',
        'identification_output_1': 'sparse_categorical_crossentropy',
        'isolation_output_1': 'sparse_categorical_crossentropy',
        'detection_output_2': 'binary_crossentropy',
        'identification_output_2': 'sparse_categorical_crossentropy',
        'isolation_output_2': 'sparse_categorical_crossentropy'
    },
    optimizer='adam',
    metrics={
        'detection_output_1': ['accuracy'],
        'identification_output_1': ['accuracy'],
        'isolation_output_1': ['accuracy'],
        'detection_output_2': ['accuracy'],
        'identification_output_2': ['accuracy'],
        'isolation_output_2': ['accuracy']
    }
)
print(model.summary())

# 3.3 model train
model.fit([X_train_head_1, X_train_head_2], [y_train_detection_1, y_train_identification_1, y_train_isolation_1,
                    y_train_detection_2, y_train_identification_2, y_train_isolation_2],
          epochs=epoch_val, batch_size=batch_size_val)

# 3.4 model test
y_pred_detection_1, y_pred_identification_1, y_pred_isolation_1, \
y_pred_detection_2, y_pred_identification_2, y_pred_isolation_2 = model.predict([X_test_head_1, X_test_head_2])

# Convert predictions for binary and multi-class
y_pred_detection_1 = (y_pred_detection_1 > 0.5).astype(int).reshape(-1)
y_pred_identification_1 = np.argmax(y_pred_identification_1, axis=1)
y_pred_isolation_1 = np.argmax(y_pred_isolation_1, axis=1)
y_pred_detection_2 = (y_pred_detection_2 > 0.5).astype(int).reshape(-1)
y_pred_identification_2 = np.argmax(y_pred_identification_2, axis=1)
y_pred_isolation_2 = np.argmax(y_pred_isolation_2, axis=1)

# 3.5 print results for each column

# quad 1
accuracy = accuracy_score(y_test_detection_1, y_pred_detection_1)
precision = precision_score(y_test_detection_1, y_pred_detection_1)
recall = recall_score(y_test_detection_1, y_pred_detection_1)
f1 = f1_score(y_test_detection_1, y_pred_detection_1)

print('Quad 1:')
print(' Detection:')
print(f'    Accuracy: {accuracy:.3f}')
print(f'    Precision: {precision:.3f}')
print(f'    Recall: {recall:.3f}')
print(f'    F1-score: {f1:.3f}')

# 3.6 print results for identification
accuracy_id = accuracy_score(y_test_identification_1, y_pred_identification_1)
print(' Identification:')
print(f'    Accuracy: {accuracy_id:.3f}')

# 3.7 print results for localization
accuracy_iso = accuracy_score(y_test_isolation_1, y_pred_isolation_1)
print(' Localization:')
print(f'    Accuracy: {accuracy_iso:.3f}')


# quad 2
accuracy = accuracy_score(y_test_detection_2, y_pred_detection_2)
precision = precision_score(y_test_detection_2, y_pred_detection_2)
recall = recall_score(y_test_detection_2, y_pred_detection_2)
f1 = f1_score(y_test_detection_2, y_pred_detection_2)

print('Quad 2:')
print(' Detection:')
print(f'    Accuracy: {accuracy:.3f}')
print(f'    Precision: {precision:.3f}')
print(f'    Recall: {recall:.3f}')
print(f'    F1-score: {f1:.3f}')

# 3.6 print results for identification
accuracy_id = accuracy_score(y_test_identification_2, y_pred_identification_2)
print(' Identification:')
print(f'    Accuracy: {accuracy_id:.3f}')

# 3.7 print results for localization
accuracy_iso = accuracy_score(y_test_isolation_2, y_pred_isolation_2)
print(' Localization:')
print(f'    Accuracy: {accuracy_iso:.3f}')